# Request data from SMHI REST API

In [ ]:
import requests
from d2e2f import request_water_depth
import pandas as pd
import re
from d2e2f.visualization import visualize
import folium

In [ ]:
station = 2110  ## (Stenungsund)
request_water_depth.get(station=station)

In [ ]:
url=r"https://opendata-download-ocobs.smhi.se/"
response = requests.get(f"{url}api/version/latest/parameter/6/station/{station}/period/latest-day/data.xml")

In [ ]:
response = requests.get(f"{url}api/version/latest.json")
j = response.json()
df_parameters = pd.DataFrame()

for parameter in j['resource']:
    interesting = ['key','title','summary']
    s = pd.Series(dtype='object')
    for key in interesting:
        s[key] = parameter[key]
    
    df_parameters = df_parameters.append(s, ignore_index=True)
    
df_parameters.set_index('key', inplace=True)
df_parameters.index = df_parameters.index.astype(int)
df_parameters.sort_index(inplace=True)

In [ ]:
df_parameters

In [ ]:
response = requests.get(f"{url}api/version/latest/parameter/3.json")
j = response.json()

In [ ]:
stations = j['station']
df_stations = pd.DataFrame()

for station in stations:
    df_stations = df_stations.append(station, ignore_index=True)
df_stations['updated'] = pd.to_datetime(df_stations['updated'], unit='ms')
df_stations['active'] = df_stations['active'].replace({0:False,1:True})

In [ ]:
df_stations.head()

In [ ]:
df_stations.head()

In [ ]:
station = df_stations.iloc[0]

In [ ]:
s = station.summary
s

In [ ]:
def lat_lon(s):
    groups = re.search(r'Latitud: ([^ ]+) Longitud: ([^ ]+)',s).groups()
    output = pd.Series()    
    output['latitude'] = float(groups[0])
    output['longitude'] = float(groups[1])
    return output

In [ ]:
df_stations[['latitude','longitude']] = df_stations['summary'].apply(func=lat_lon)

In [ ]:
f, my_map = visualize.get_map(df_stations, zoom_start=4)

for index, station in df_stations.iterrows():
    color = 'green' if station.active else 'red'
    folium.Marker(
                [station["latitude"], station["longitude"]],
                popup=station[['key','name','title','updated']],
                icon=folium.Icon(color=color, icon="stream"),
            ).add_to(my_map)

display(f)

In [ ]:
from io import StringIO

def get_data(key = 35110, parameter_id = 3):
    response = requests.get(f"{url}api/version/latest/parameter/{parameter_id}/station/{key}/period/corrected-archive/data.csv")
    
    assert response.ok
    
    s = response.text
    
    df_current = pd.read_csv(StringIO(s), sep=';', header=4, index_col=0)
    df_current.index = pd.to_datetime(df_current.index)
    
    return df_current

In [ ]:
df_current = get_data(key=35110, parameter_id=3)
df_current.head()

In [ ]:
df_current = get_data(key=35110, parameter_id=2)
df_current.head()

In [ ]:
df_current.tail()

In [ ]:
df_current = get_data(key=38043, parameter_id=2)
df_current.tail()